In [1]:
import csv
from public_data.storages import DataStorage
from prettytable import PrettyTable
from public_data.models import CouvertureSol, UsageSol, CouvertureUsageMatrix

filename = "Artif_conso_NAF.csv"
storage = DataStorage()


In [2]:
file_stream = storage.open(filename)
lines = file_stream.read()
lines = lines.decode('utf-8-sig')
lines = lines.splitlines(True)
lines[0] = lines[0].replace(" ", ".")
lines

[';CS1.1.1.1;CS1.1.1.2;CS1.1.2.1;CS1.1.2.2;CS1.2.1;CS1.2.2;CS1.2.3;CS2.1.1.1;CS2.1.1.2;CS2.1.1.3;CS2.1.2;CS2.1.3;CS2.2.1;CS2.2.2\r\n',
 'US1.1;Artif non conso;Artif;Artif;;;NAF;;NAF;;;NAF;NAF;NAF;NAF\r\n',
 'US1.2;Artif non conso;Artif;Artif;;;;;NAF;NAF;NAF;NAF;;;\r\n',
 'US1.3;Artif;Artif;Artif;;;NAF;;NAF;NAF;NAF;NAF;;NAF;NAF\r\n',
 'US1.4;Artif;Artif;Artif;;;NAF;;NAF;NAF;NAF;NAF;;NAF;\r\n',
 'US1.5;Artif;Artif;Artif;;;;;NAF;NAF;NAF;NAF;;NAF;\r\n',
 'US235;Artif;Artif;Artif;Artif;NAF;NAF;;Conso;Conso;Conso;Conso;Conso;Conso;\r\n',
 'US2;Artif;Artif;Artif;;NAF;NAF;;Conso;Conso;Conso;Conso;Conso;Conso;\r\n',
 'US3;Artif;Artif;Artif;Artif;NAF;NAF;;Conso;Conso;Conso;Conso;Conso;Conso;\r\n',
 'US5;Artif;Artif;Artif;Artif;NAF;NAF;;Conso;Conso;Conso;Conso;Conso;Conso;\r\n',
 'US4.1.1;Artif;Artif;Artif;;;NAF;;Conso;Conso;Conso;Conso;Conso;Conso;Conso\r\n',
 'US4.1.2;Artif;Artif;Artif;;;;;Conso;Conso;Conso;Conso;Conso;Conso;Conso\r\n',
 'US4.1.3;Artif;Artif;Artif;;;;;Conso;Conso;Conso;Conso;;C

In [3]:
headers = lines[0].split(";")
headers = [_.replace("\r\n", "") for _ in headers if _]
headers

['CS1.1.1.1',
 'CS1.1.1.2',
 'CS1.1.2.1',
 'CS1.1.2.2',
 'CS1.2.1',
 'CS1.2.2',
 'CS1.2.3',
 'CS2.1.1.1',
 'CS2.1.1.2',
 'CS2.1.1.3',
 'CS2.1.2',
 'CS2.1.3',
 'CS2.2.1',
 'CS2.2.2']

In [4]:
reader = csv.DictReader(lines, delimiter=';')
items = []
for row in reader:
    usage_code = row['']
    for couv_code in headers:
        if row[couv_code]:
            items.append((usage_code, couv_code, row[couv_code]))

items.append(("US2", "CS1.1.2.2", "Artif"))            

print(len(items))
items[:10]

192


[('US1.1', 'CS1.1.1.1', 'Artif non conso'),
 ('US1.1', 'CS1.1.1.2', 'Artif'),
 ('US1.1', 'CS1.1.2.1', 'Artif'),
 ('US1.1', 'CS1.2.2', 'NAF'),
 ('US1.1', 'CS2.1.1.1', 'NAF'),
 ('US1.1', 'CS2.1.2', 'NAF'),
 ('US1.1', 'CS2.1.3', 'NAF'),
 ('US1.1', 'CS2.2.1', 'NAF'),
 ('US1.1', 'CS2.2.2', 'NAF'),
 ('US1.2', 'CS1.1.1.1', 'Artif non conso')]

In [5]:
def get_all_children(cs):
    children = []
    for cs_child in cs.children.all():
        children += get_all_children(cs_child)
        children.append(cs_child)
    return children

for usage_code, couv_code, label in items:
    cs = CouvertureSol.objects.get(code_prefix=couv_code)
    us = UsageSol.objects.get(code_prefix=usage_code)
    all_cs = get_all_children(cs) + [cs]
    all_us = get_all_children(us) + [us]
    all = list(zip(all_cs, all_us))
    if len(all) > 1:
        print(usage_code, couv_code, label)
        print(all)
len(items)    

192

In [6]:
for usage_code, couv_code, label in items:
    try:
        matrix = CouvertureUsageMatrix.objects.get(
            couverture__code_prefix=couv_code, usage__code_prefix=usage_code
        )
    except CouvertureUsageMatrix.DoesNotExist:
        print(usage_code, couv_code, label)

In [7]:
labels = set([item[2] for item in items])
labels

{'Artif', 'Artif non conso', 'Conso', 'NAF'}

In [9]:
transco = {
    'Artif': CouvertureUsageMatrix.LabelChoices.ARTIFICIAL,
    'Artif non conso': CouvertureUsageMatrix.LabelChoices.ARTIF_NOT_CONSUMED,
    'Conso': CouvertureUsageMatrix.LabelChoices.CONSUMED,
    'NAF': CouvertureUsageMatrix.LabelChoices.NAF,
}
transco

{'Artif': <LabelChoices.ARTIFICIAL: 'ARTIF'>,
 'Artif non conso': <LabelChoices.ARTIF_NOT_CONSUMED: 'ARTIF_NOT_CONSU'>,
 'Conso': <LabelChoices.CONSUMED: 'CONSU'>,
 'NAF': <LabelChoices.NAF: 'NAF'>}

In [10]:
for usage_code, couv_code, label in items:
    matrix = CouvertureUsageMatrix.objects.get(couverture__code_prefix=couv_code, usage__code_prefix=usage_code)
    matrix.label = transco[label]
    matrix.compute()
    print(label, matrix)
    matrix.save()

Artif non conso CS1.1.1.1-US1.1:a
Artif CS1.1.1.2-US1.1:ac
Artif CS1.1.2.1-US1.1:ac
NAF CS1.2.2-US1.1:n
NAF CS2.1.1.1-US1.1:n
NAF CS2.1.2-US1.1:n
NAF CS2.1.3-US1.1:n
NAF CS2.2.1-US1.1:n
NAF CS2.2.2-US1.1:n
Artif non conso CS1.1.1.1-US1.2:a
Artif CS1.1.1.2-US1.2:ac
Artif CS1.1.2.1-US1.2:ac
NAF CS2.1.1.1-US1.2:n
NAF CS2.1.1.2-US1.2:n
NAF CS2.1.1.3-US1.2:n
NAF CS2.1.2-US1.2:n
Artif CS1.1.1.1-US1.3:ac
Artif CS1.1.1.2-US1.3:ac
Artif CS1.1.2.1-US1.3:ac
NAF CS1.2.2-US1.3:n
NAF CS2.1.1.1-US1.3:n
NAF CS2.1.1.2-US1.3:n
NAF CS2.1.1.3-US1.3:n
NAF CS2.1.2-US1.3:n
NAF CS2.2.1-US1.3:n
NAF CS2.2.2-US1.3:n
Artif CS1.1.1.1-US1.4:ac
Artif CS1.1.1.2-US1.4:ac
Artif CS1.1.2.1-US1.4:ac
NAF CS1.2.2-US1.4:n
NAF CS2.1.1.1-US1.4:n
NAF CS2.1.1.2-US1.4:n
NAF CS2.1.1.3-US1.4:n
NAF CS2.1.2-US1.4:n
NAF CS2.2.1-US1.4:n
Artif CS1.1.1.1-US1.5:ac
Artif CS1.1.1.2-US1.5:ac
Artif CS1.1.2.1-US1.5:ac
NAF CS2.1.1.1-US1.5:n
NAF CS2.1.1.2-US1.5:n
NAF CS2.1.1.3-US1.5:n
NAF CS2.1.2-US1.5:n
NAF CS2.2.1-US1.5:n
Artif CS1.1.1.1-US235